# 假设文档嵌入 (HyDE) 用于 RAG

在这个笔记本中，我实现了 HyDE（假设文档嵌入）- 一种创新的检索技术，它在执行检索之前将用户查询转换为假设的答案文档。这种方法弥合了短查询和长文档之间的语义差距。

传统的 RAG 系统直接嵌入用户的短查询，但这往往无法捕获最佳检索所需的语义丰富性。HyDE 通过以下方式解决这个问题：

- 生成一个回答查询的假设文档
- 嵌入这个扩展的文档而不是原始查询
- 检索与这个假设文档相似的文档
- 创建更具上下文相关性的答案

## 设置环境
我们首先导入必要的库。

In [1]:
import os
import numpy as np
import json
import fitz
from openai import OpenAI
import re
import matplotlib.pyplot as plt

## 设置 OpenAI API 客户端
我们初始化 OpenAI 客户端来生成嵌入向量和响应。

In [ ]:
# 使用基础 URL 和 API 密钥初始化 OpenAI 客户端
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
)

## 文档处理函数

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本内容，并按页面分离。
    
    Args:
        pdf_path (str): PDF 文件路径
        
    Returns:
        List[Dict]: 包含文本内容和元数据的页面列表
    """
    print(f"正在从 {pdf_path} 提取文本...")  # 打印正在处理的 PDF 路径
    pdf = fitz.open(pdf_path)  # 使用 PyMuPDF 打开 PDF 文件
    pages = []  # 初始化空列表来存储包含文本内容的页面
    
    # 遍历 PDF 中的每一页
    for page_num in range(len(pdf)):
        page = pdf[page_num]  # 获取当前页面
        text = page.get_text()  # 从当前页面提取文本
        
        # 跳过文本很少的页面（少于 50 个字符）
        if len(text.strip()) > 50:
            # 将页面文本和元数据追加到列表中
            pages.append({
                "text": text,
                "metadata": {
                    "source": pdf_path,  # 源文件路径
                    "page": page_num + 1  # 页码（从 1 开始的索引）
                }
            })
    
    print(f"提取了 {len(pages)} 页内容")  # 打印提取的页面数量
    return pages  # 返回包含文本内容和元数据的页面列表

In [4]:
def chunk_text(text, chunk_size=1000, overlap=200):
    """
    将文本分割为重叠的块。
    
    Args:
        text (str): 要分块的输入文本
        chunk_size (int): 每个块的字符大小
        overlap (int): 块之间的重叠字符数
        
    Returns:
        List[Dict]: 包含元数据的块列表
    """
    chunks = []  # 初始化空列表来存储块
    
    # 以 (chunk_size - overlap) 的步长遍历文本
    for i in range(0, len(text), chunk_size - overlap):
        chunk_text = text[i:i + chunk_size]  # 提取文本块
        if chunk_text:  # 确保不添加空块
            chunks.append({
                "text": chunk_text,  # 添加块文本
                "metadata": {
                    "start_pos": i,  # 块在原始文本中的起始位置
                    "end_pos": i + len(chunk_text)  # 块在原始文本中的结束位置
                }
            })
    
    print(f"创建了 {len(chunks)} 个文本块")  # 打印创建的块数量
    return chunks  # 返回包含元数据的块列表

## 简单向量存储实现

In [5]:
class SimpleVectorStore:
    """
    使用 NumPy 的简单向量存储实现。
    """
    def __init__(self):
        self.vectors = []  # 存储向量嵌入的列表
        self.texts = []  # 存储文本内容的列表
        self.metadata = []  # 存储元数据的列表
    
    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加项目。
        
        Args:
            text (str): 文本内容
            embedding (List[float]): 向量嵌入
            metadata (Dict, optional): 附加元数据
        """
        self.vectors.append(np.array(embedding))  # 将嵌入作为 numpy 数组追加
        self.texts.append(text)  # 追加文本内容
        self.metadata.append(metadata or {})  # 追加元数据或空字典（如果为 None）
    
    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        查找与查询嵌入最相似的项目。
        
        Args:
            query_embedding (List[float]): 查询嵌入向量
            k (int): 要返回的结果数量
            filter_func (callable, optional): 过滤结果的函数
            
        Returns:
            List[Dict]: 前 k 个最相似的项目
        """
        if not self.vectors:
            return []  # 如果没有向量，返回空列表
        
        # 将查询嵌入转换为 numpy 数组
        query_vector = np.array(query_embedding)
        
        # 使用余弦相似度计算相似性
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 如果不通过过滤器则跳过
            if filter_func and not filter_func(self.metadata[i]):
                continue
                
            # 计算余弦相似度
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 追加索引和相似度分数
        
        # 按相似度排序（降序）
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 返回前 k 个结果
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 添加文本内容
                "metadata": self.metadata[idx],  # 添加元数据
                "similarity": float(score)  # 添加相似度分数
            })
        
        return results  # 返回前 k 个结果列表

## 创建嵌入向量

In [6]:
def create_embeddings(texts, model="BAAI/bge-en-icl"):
    """
    为给定的文本创建嵌入向量。
    
    Args:
        texts (List[str]): 输入文本
        model (str): 嵌入模型名称
        
    Returns:
        List[List[float]]: 嵌入向量
    """
    # 处理空输入
    if not texts:
        return []
        
    # 如果需要，分批处理（OpenAI API 限制）
    batch_size = 100
    all_embeddings = []
    
    # 分批遍历输入文本
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]  # 获取当前批次的文本
        
        # 为当前批次创建嵌入向量
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 将批次嵌入向量添加到列表中
    
    return all_embeddings  # 返回所有嵌入向量

## 文档处理流水线

In [7]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    为 RAG 处理文档。
    
    Args:
        pdf_path (str): PDF 文件路径
        chunk_size (int): 每个块的字符大小
        chunk_overlap (int): 块之间的重叠字符数
        
    Returns:
        SimpleVectorStore: 包含文档块的向量存储
    """
    # 从 PDF 文件中提取文本
    pages = extract_text_from_pdf(pdf_path)
    
    # 处理每个页面并创建块
    all_chunks = []
    for page in pages:
        # 将文本内容（字符串）传递给 chunk_text，而不是字典
        page_chunks = chunk_text(page["text"], chunk_size, chunk_overlap)
        
        # 用页面的元数据更新每个块的元数据
        for chunk in page_chunks:
            chunk["metadata"].update(page["metadata"])
        
        all_chunks.extend(page_chunks)
    
    # 为文本块创建嵌入向量
    print("正在为块创建嵌入向量...")
    chunk_texts = [chunk["text"] for chunk in all_chunks]
    chunk_embeddings = create_embeddings(chunk_texts)
    
    # 创建向量存储来保存块及其嵌入向量
    vector_store = SimpleVectorStore()
    for i, chunk in enumerate(all_chunks):
        vector_store.add_item(
            text=chunk["text"],
            embedding=chunk_embeddings[i],
            metadata=chunk["metadata"]
        )
    
    print(f"创建了包含 {len(all_chunks)} 个块的向量存储")
    return vector_store

## 假设文档生成

In [8]:
def generate_hypothetical_document(query, desired_length=1000):
    """
    生成一个回答查询的假设文档。
    
    Args:
        query (str): 用户查询
        desired_length (int): 假设文档的目标长度
        
    Returns:
        str: 生成的假设文档
    """
    # 定义指导模型如何生成文档的系统提示
    system_prompt = f"""您是一位专业的文档创建者。
    给定一个问题，生成一个能够直接回答这个问题的详细文档。
    文档应该大约 {desired_length} 个字符长，并提供对问题的深入、
    信息丰富的答案。写作时就像这个文档来自该主题的权威来源。
    包含具体的细节、事实和解释。
    不要提及这是一个假设文档 - 直接写内容即可。"""

    # 定义包含查询的用户提示
    user_prompt = f"问题：{query}\n\n生成一个完全回答这个问题的文档："
    
    # 向 OpenAI API 发出请求以生成假设文档
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",  # 指定要使用的模型
        messages=[
            {"role": "system", "content": system_prompt},  # 指导助手的系统消息
            {"role": "user", "content": user_prompt}  # 包含查询的用户消息
        ],
        temperature=0.1  # 设置响应生成的温度
    )
    
    # 返回生成的文档内容
    return response.choices[0].message.content

## 完整的 HyDE RAG 实现

In [9]:
def hyde_rag(query, vector_store, k=5, should_generate_response=True):
    """
    使用假设文档嵌入执行 RAG。
    
    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 包含文档块的向量存储
        k (int): 要检索的块数量
        should_generate_response (bool): 是否生成最终响应
        
    Returns:
        Dict: 包括假设文档和检索块的结果
    """
    print(f"\n=== 使用 HyDE 处理查询：{query} ===\n")
    
    # 步骤 1：生成一个回答查询的假设文档
    print("正在生成假设文档...")
    hypothetical_doc = generate_hypothetical_document(query)
    print(f"生成了 {len(hypothetical_doc)} 个字符的假设文档")
    
    # 步骤 2：为假设文档创建嵌入向量
    print("正在为假设文档创建嵌入向量...")
    hypothetical_embedding = create_embeddings([hypothetical_doc])[0]
    
    # 步骤 3：基于假设文档检索相似块
    print(f"正在检索 {k} 个最相似的块...")
    retrieved_chunks = vector_store.similarity_search(hypothetical_embedding, k=k)
    
    # 准备结果字典
    results = {
        "query": query,
        "hypothetical_document": hypothetical_doc,
        "retrieved_chunks": retrieved_chunks
    }
    
    # 步骤 4：如果需要，生成响应
    if should_generate_response:
        print("正在生成最终响应...")
        response = generate_response(query, retrieved_chunks)
        results["response"] = response
    
    return results

## 用于比较的标准（直接）RAG 实现

In [10]:
def standard_rag(query, vector_store, k=5, should_generate_response=True):
    """
    使用直接查询嵌入执行标准 RAG。
    
    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 包含文档块的向量存储
        k (int): 要检索的块数量
        should_generate_response (bool): 是否生成最终响应
        
    Returns:
        Dict: 包括检索块的结果
    """
    print(f"\n=== 使用标准 RAG 处理查询：{query} ===\n")
    
    # 步骤 1：为查询创建嵌入向量
    print("正在为查询创建嵌入向量...")
    query_embedding = create_embeddings([query])[0]
    
    # 步骤 2：基于查询嵌入向量检索相似块
    print(f"正在检索 {k} 个最相似的块...")
    retrieved_chunks = vector_store.similarity_search(query_embedding, k=k)
    
    # 准备结果字典
    results = {
        "query": query,
        "retrieved_chunks": retrieved_chunks
    }
    
    # 步骤 3：如果需要，生成响应
    if should_generate_response:
        print("正在生成最终响应...")
        response = generate_response(query, retrieved_chunks)
        results["response"] = response
        
    return results

## 响应生成

In [11]:
def generate_response(query, retrieved_chunks):
    """
    基于检索到的块生成响应。
    
    Args:
        query (str): 用户查询
        retrieved_chunks (List[Dict]): 检索到的文本块
        
    Returns:
        str: 生成的响应
    """
    # 从检索到的块中提取文本
    context_texts = [chunk["text"] for chunk in retrieved_chunks]
    
    # 将提取的文本合并为单个上下文字符串，用 "---" 分隔
    combined_context = "\n\n---\n\n".join(context_texts)
    
    # 定义上下文的最大允许长度（OpenAI 限制）
    max_context = 14000
    
    # 如果合并的上下文超过最大长度，则截断
    if len(combined_context) > max_context:
        combined_context = combined_context[:max_context] + "... [已截断]"
    
    # 定义指导 AI 助手的系统消息
    system_message = """您是一个有用的 AI 助手。基于提供的上下文回答用户的问题。
如果信息不在上下文中，请说明。在可能的情况下，在您的答案中引用上下文的特定部分。"""

    # 使用 OpenAI API 生成响应
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",  # 指定要使用的模型
        messages=[
            {"role": "system", "content": system_message},  # 指导助手的系统消息
            {"role": "user", "content": f"上下文：\n{combined_context}\n\n问题：{query}"}  # 包含上下文和查询的用户消息
        ],
        temperature=0.2  # 设置响应生成的温度
    )
    
    # 返回生成的响应内容
    return response.choices[0].message.content

## 评估函数

In [12]:
def compare_approaches(query, vector_store, reference_answer=None):
    """
    比较 HyDE 和标准 RAG 方法对查询的处理。
    
    Args:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 包含文档块的向量存储
        reference_answer (str, optional): 用于评估的参考答案
        
    Returns:
        Dict: 比较结果
    """
    # 运行 HyDE RAG
    hyde_result = hyde_rag(query, vector_store)
    hyde_response = hyde_result["response"]
    
    # 运行标准 RAG
    standard_result = standard_rag(query, vector_store)
    standard_response = standard_result["response"]
    
    # 比较结果
    comparison = compare_responses(query, hyde_response, standard_response, reference_answer)
    
    return {
        "query": query,
        "hyde_response": hyde_response,
        "hyde_hypothetical_doc": hyde_result["hypothetical_document"],
        "standard_response": standard_response,
        "reference_answer": reference_answer,
        "comparison": comparison
    }

In [13]:
def compare_responses(query, hyde_response, standard_response, reference=None):
    """
    比较 HyDE 和标准 RAG 的响应。
    
    Args:
        query (str): 用户查询
        hyde_response (str): HyDE RAG 的响应
        standard_response (str): 标准 RAG 的响应
        reference (str, optional): 参考答案
        
    Returns:
        str: 比较分析
    """
    system_prompt = """您是信息检索系统的专家评估员。
比较对同一查询的两个响应，一个使用 HyDE（假设文档嵌入）生成，
另一个使用直接查询嵌入的标准 RAG 生成。

基于以下方面评估它们：
1. 准确性：哪个响应提供了更多事实正确的信息？
2. 相关性：哪个响应更好地解决了查询？
3. 完整性：哪个响应提供了更全面的主题覆盖？
4. 清晰度：哪个响应组织得更好，更容易理解？

在分析每种方法的优缺点时要具体。"""

    user_prompt = f"""查询：{query}

HyDE RAG 的响应：
{hyde_response}

标准 RAG 的响应：
{standard_response}"""

    if reference:
        user_prompt += f"""

参考答案：
{reference}"""

    user_prompt += """

请提供这两个响应的详细比较，突出哪种方法表现更好以及原因。"""

    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    return response.choices[0].message.content

## 示例使用和评估

以下代码演示了如何使用 HyDE RAG 系统并与标准 RAG 进行比较。

In [14]:
# 示例使用
if __name__ == "__main__":
    # PDF 文档路径
    pdf_path = "data/AI_Information.pdf"
    
    # 测试查询
    query = "人工智能开发中的主要伦理考虑因素有哪些？"
    
    # 处理文档并创建向量存储
    vector_store = process_document(pdf_path)
    
    # 运行 HyDE RAG
    hyde_result = hyde_rag(query, vector_store)
    
    print(f"查询：{hyde_result['query']}")
    print(f"假设文档长度：{len(hyde_result['hypothetical_document'])} 个字符")
    print(f"检索的块数量：{len(hyde_result['retrieved_chunks'])}")
    print(f"HyDE 响应：{hyde_result['response']}")
    
    # 运行标准 RAG 进行比较
    standard_result = standard_rag(query, vector_store)
    print(f"\n标准 RAG 响应：{standard_result['response']}")
    
    # 比较两种方法
    comparison = compare_approaches(query, vector_store)
    print(f"\n比较分析：{comparison['comparison']}")

## 总结

假设文档嵌入 (HyDE) 通过以下方式改进了传统的 RAG 系统：

1. **语义桥接**：通过生成假设文档来弥合短查询和长文档之间的语义差距
2. **上下文扩展**：将简短的查询扩展为丰富的假设答案，提供更好的检索上下文
3. **改进的匹配**：假设文档通常与实际相关文档具有更好的语义相似性
4. **查询理解**：通过生成过程更好地理解查询意图

这种方法特别适用于：
- 复杂或抽象的查询
- 需要深度理解的问题
- 查询和文档之间存在词汇差距的情况
- 需要综合多个概念的查询

HyDE 代表了检索增强生成技术的重要进步，为构建更智能、更具上下文感知能力的问答系统提供了强大的框架。通过首先生成假设答案然后检索相关内容，HyDE 能够更好地理解用户意图并提供更准确的响应。